In [1]:
import numpy as np
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Activation, BatchNormalization, Conv2D

In [2]:
class IdentityBlock(keras.layers.Layer):
  def __init__(self, channels, f):
    super(IdentityBlock, self).__init__(name='IdentityBlock')
    self.channels = channels
    self.f = f

  def build(self, input):
    self.conv1 = Conv2D(filters = self.channels[0], kernel_size = (1,1), strides = (1,1), \
                        padding ='valid')
    self.bn1 = BatchNormalization(axis = 3)
    self.conv2 = Conv2D(filters = self.channels[1], kernel_size = (self.f,self.f), strides = (1,1),\
                        padding ='same')
    self.bn2 = BatchNormalization(axis = 3)
    self.conv3 = Conv2D(filters = self.channels[2], kernel_size = (1,1), strides = (1,1), \
                        padding ='valid')
    self.bn3 = BatchNormalization(axis = 3)
    self.relu = Activation('relu')

  def call(self, x):
    ## Main path ##
    x1 = self.conv1(x)
    x1 = self.bn1(x1)
    x1 = self.relu(x1)
    x1 = self.conv2(x1)
    x1 = self.bn2(x1)
    x1 = self.relu(x1)
    x1 = self.conv3(x1)
    x1 = self.bn3(x1)

    ## Shortcut path ##
    x1 = Add()([x, x1])
    x1 = self.relu(x1)
    print(x1)
    return x1


In [3]:
# generate some fake data to work with
np.random.seed(0)
X = np.random.randn(3, 4, 4, 6)
print('original data shape:', X.shape)

# Defining parameters to pass in the function
input = Input(shape=X.shape[1:])
identity_block_obj = IdentityBlock(channels = [2, 4, 6], f = 3)
outputs = identity_block_obj(input)
print(outputs.shape)

original data shape: (3, 4, 4, 6)
Tensor("IdentityBlock/activation/Relu_2:0", shape=(None, 4, 4, 6), dtype=float32)
(None, 4, 4, 6)


In [4]:
class ConvolutionalBlock(keras.layers.Layer):
  def __init__(self, channels, f, strides):
    super(ConvolutionalBlock, self).__init__(name='ConvolutionalBlock')
    self.channels = channels
    self.f = f
    self.s = strides

  def build(self, input):
    self.conv1 = Conv2D(filters = self.channels[0], kernel_size = (1,1), \
                        strides = (self.s,self.s), padding ='valid')
    self.bn1 = BatchNormalization(axis = 3)
    self.conv2 = Conv2D(filters = self.channels[1], kernel_size = (self.f,self.f),\
                        strides = (1,1), padding ='same')
    self.bn2 = BatchNormalization(axis = 3)
    self.conv3 = Conv2D(filters = self.channels[2], kernel_size = (1,1), \
                        strides = (1,1), padding ='valid')
    self.bn3 = BatchNormalization(axis = 3)
    self.conv_shortcut = Conv2D(filters = self.channels[2], kernel_size = (1, 1),\
                                strides = (self.s,self.s), padding = 'valid')
    self.bn4= BatchNormalization(axis = 3)

    self.relu = Activation('relu')

  def call(self, x):
    ## Main path ##
    x1 = self.conv1(x)
    x1 = self.bn1(x1)
    x1 = self.relu(x1)
    x1 = self.conv2(x1)
    x1 = self.bn2(x1)
    x1 = self.relu(x1)
    x1 = self.conv3(x1)
    x1 = self.bn3(x1)

    ## Shortcut path ##
    x2 = self.conv_shortcut(x)
    x2 = self.bn4(x2)

    ## Adding output of 2 paths ##
    x1 = Add()([x2, x1])
    x1 = self.relu(x1)

    return x1

In [5]:
# generate some fake data to work with
np.random.seed(0)
X = np.random.randn(3, 4, 4, 6)
print('original data shape:', X.shape)

# Defining parameters to pass in the function
input = layers.Input(shape=X.shape[1:])
conv_block_obj = ConvolutionalBlock(channels = [2, 4, 6], f = 3, strides = 2)
outputs = conv_block_obj(input)
print(outputs.shape)

original data shape: (3, 4, 4, 6)
(None, 2, 2, 6)
